In [1]:
import tensorflow as tf

In [2]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

In [3]:
x_val = x_train[50000: 60000]
x_train = x_train[: 50000]
y_val = y_train[50000: 60000]
y_train = y_train[: 50000]
print('train data has ', str(x_train.shape[0]), 'samples')
print('every train data is ', str(x_train.shape[1]), '*', str(x_train.shape[2]), 'image')

train data has  50000 samples
every train data is  28 * 28 image


In [4]:
print('validation data has ', str(x_val.shape[0]), 'samples')
print('every train data is ', str(x_val.shape[1]), '*', str(x_val.shape[2]), 'image')

validation data has  10000 samples
every train data is  28 * 28 image


In [5]:
import numpy as np

In [6]:
x_train = np.reshape(x_train, (50000, 28, 28, 1))
x_val = np.reshape(x_val, (10000, 28, 28, 1))
x_test = np.reshape(x_test, (10000, 28, 28, 1))

print(x_train.shape)

(50000, 28, 28, 1)


In [7]:
print(x_test.shape)

(10000, 28, 28, 1)


In [8]:
x_train = x_train.astype('float32')
x_val = x_val.astype('float32')
x_test = x_test.astype('float32')
gray_scale = 255
x_train /= gray_scale
x_val /= gray_scale
x_test /= gray_scale

In [9]:
num_classes = 10
y_train = tf.keras.utils.to_categorical(y_train, num_classes)
y_val = tf.keras.utils.to_categorical(y_val, num_classes)
y_test = tf.keras.utils.to_categorical(y_test, num_classes)

In [10]:
x = tf.placeholder(tf.float32, shape= [None, 28, 28, 1])
y_ = tf.placeholder(tf.float32, shape= [None, 10])

In [11]:
def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev= 0.1)
    return tf.Variable(initial)

In [12]:
def bias_variable(shape):
    initial = tf.constant(0.1, shape= shape)
    return tf.Variable(initial)

In [13]:
def conv2d(x, w):
    return tf.nn.conv2d(x, w, strides= [1, 1, 1, 1], padding= 'SAME')

In [14]:
def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize= [1, 2, 2, 1], strides= [1, 2, 2, 1], padding= 'SAME')

In [15]:
w_conv1 = weight_variable([5, 5, 1, 16])
b_conv1 = bias_variable([16])

In [16]:
h_conv1 = tf.nn.relu(conv2d(x, w_conv1) + b_conv1)

In [17]:
h_pool1 = max_pool_2x2(h_conv1)

In [18]:
w_conv2 = weight_variable([5, 5, 16, 32])
b_conv2 = bias_variable([32])
h_conv2 = tf.nn.relu(conv2d(h_pool1, w_conv2) + b_conv2)
h_pool2 = max_pool_2x2(h_conv2)

In [19]:
w_fc1 = weight_variable([7 * 7 * 32, 128])
b_fc1 = bias_variable([128])

h_pool2_flat = tf.reshape(h_pool2, [-1, 7 * 7 * 32])
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, w_fc1) + b_fc1)

In [20]:
w_fc2 = weight_variable([128, 10])
b_fc2 = bias_variable([10])
y_conv = tf.matmul(h_fc1, w_fc2) + b_fc2

In [21]:
cross_entropy = tf.reduce_mean(
tf.nn.softmax_cross_entropy_with_logits_v2(labels= y_, logits= y_conv))

In [22]:
train_step = tf.train.AdamOptimizer(0.001).minimize(cross_entropy)

In [23]:
correct_prediction = tf.equal(tf.argmax(y_conv, 1), tf.argmax(y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [24]:
init = tf.global_variables_initializer()

epoch_cnt = 3
batch_size = 500
iteration = len(x_train) //  batch_size

with tf.Session() as sess:
    tf.set_random_seed(777)
    
    sess.run(init)
    for epoch in range(epoch_cnt):
        avg_loss = 0
        start = 0
        end = batch_size
        
        for i in range(iteration):
            if i % 10 == 0:
                train_acc = accuracy.eval(
                    feed_dict = {x: x_train[start: end],
                                 y_ : y_train[start: end]})
                print('step ', str(i), ': train acc: ', str(train_acc))
            
            train_step.run(
                feed_dict = {x: x_train[start: end],
                             y_ : y_train[start: end]})
            start += batch_size
            end += batch_size
            
            val_accuracy = accuracy.eval(
                feed_dict = {x: x_val[start: end],
                             y_ : y_val[start: end]})
            print('validation accuracy: ', str(val_accuracy))
        
        test_accuracy = accuracy.eval(
            feed_dict = {x: x_test,
                         y_ : y_test})
        print('test accuracy: ', str(test_accuracy))

step  0 : train acc:  0.084
validation accuracy:  0.132
validation accuracy:  0.334
validation accuracy:  0.362
validation accuracy:  0.456
validation accuracy:  0.58
validation accuracy:  0.672
validation accuracy:  0.688
validation accuracy:  0.67
validation accuracy:  0.664
validation accuracy:  0.756
step  10 : train acc:  0.698
validation accuracy:  0.786
validation accuracy:  0.792
validation accuracy:  0.81
validation accuracy:  0.84
validation accuracy:  0.816
validation accuracy:  0.888
validation accuracy:  0.89
validation accuracy:  0.886
validation accuracy:  0.89
validation accuracy:  nan
step  20 : train acc:  0.846
validation accuracy:  nan
validation accuracy:  nan
validation accuracy:  nan
validation accuracy:  nan
validation accuracy:  nan
validation accuracy:  nan
validation accuracy:  nan
validation accuracy:  nan
validation accuracy:  nan
validation accuracy:  nan
step  30 : train acc:  0.87
validation accuracy:  nan
validation accuracy:  nan
validation accuracy:  

validation accuracy:  nan
validation accuracy:  nan
validation accuracy:  nan
validation accuracy:  nan
validation accuracy:  nan
validation accuracy:  nan
validation accuracy:  nan
step  90 : train acc:  0.976
validation accuracy:  nan
validation accuracy:  nan
validation accuracy:  nan
validation accuracy:  nan
validation accuracy:  nan
validation accuracy:  nan
validation accuracy:  nan
validation accuracy:  nan
validation accuracy:  nan
validation accuracy:  nan
test accuracy:  0.9763
